## **Imports and configurations**

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import time

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import plotly.graph_objs as go

from IPython.display import IFrame
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

from plot_utils import *

In [3]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier

from sklearn.utils import shuffle
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

In [5]:
folder_path = 'C:/Users/yaass/OneDrive/Bureau/Parser'

## **Models evaluation**

In [6]:
def preview_data(folder_path, file_name, index_value = 'md5'):
    df = pd.read_csv(os.path.join(folder_path, file_name), index_col=index_value)
    print(f'Nb Observations: {df.shape[0]}')
    print(f'Nb Columns: {df.shape[1]}')
    return df.head(3)


def create_X_y(folder_path, file_name, index_value = 'md5'):
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path, index_col = index_value)
    X = df.drop('label', axis=1)
    y = df['label']
    return shuffle(X, y)


def create_regular_net():
    model = Sequential()
    model.add(Dense(units=10, kernel_initializer = 'uniform', activation = 'relu', name='dense_layer1'))
    model.add(Dense(units=10, kernel_initializer = 'uniform', activation = 'relu', name='dense_layer2'))
    model.add(Dense(1, activation = 'sigmoid', name = 'dense_output'))   #sigmoid for binary 
    model.compile(loss='binary_crossentropy', optimizer= 'adam', metrics = ['accuracy'])
    return model


def wrap_regular_net():
    model = KerasClassifier(build_fn=create_regular_net, epochs=50, batch_size=64, verbose=0)
    return model


def create_models():
    models = dict()
    models['LogisticRegression'] = LogisticRegression()
    models['KNN'] = KNeighborsClassifier()
    models['Decision tree'] = DecisionTreeClassifier()
    models['Random Forest'] = RandomForestClassifier()
    #models['Stochastic Gradient Descent'] = SGDClassifier()
    models['SVM'] = SVC()
    models['RegularNets'] = wrap_regular_net()
    models['LDA'] = LDA()
    models['Gaussian Naive Bayes'] = GaussianNB()
    return models


def evaluate_model(model, X, y):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
    return scores


def get_evaluation_results(folder_path, file_name):  
    
    #generate features and target
    X, y = create_X_y(folder_path, file_name)
    
    #generate models
    models = create_models()
    
    #evaluate models and store results
    results, names = list(), list()
    for name, model in models.items():
        scores = evaluate_model(model, X, y)
        results.append(scores)
        names.append(name) 
        
    return names, results


def print_evaluation_results(results, names):  
    for name, scores in zip(names, results):
        print(f"{name:30} \t: {np.mean(scores)*100:.3f}% ( {np.std(scores)*100:.3f}% )")  

### 1. apistats (one-hot encoded)

In [9]:
file_name = 'onehot_encoded_apistats_dataset.csv'

preview_data(folder_path, file_name)

Nb Observations: 3761
Nb Columns: 304


,LdrUnloadDll,RegCloseKey,GetSystemTimeAsFileTime,LoadStringW,GetSystemInfo,RegQueryValueExA,LdrGetProcedureAddress,MessageBoxTimeoutW,RegSetValueExA,NtTerminateProcess,...,CIFrameElement_CreateElement,CDocument_write,WSAConnect,CopyFileExW,NtDeleteFile,ExitWindowsEx,CreateServiceW,WNetGetProviderNameW,RtlCreateUserProcess,NtLoadKeyEx
md5,,,,,,,,,,,,,,,,,,,,,
00263ca2071dc9a6ee577eb356b0d1d9,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
0137996cd3aa197ae8eb64fef12c044a,0,0,1,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
0282f83bbfb58c08b54dbd8015e54d2e,0,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
start = time.time()

names, results = get_evaluation_results(folder_path, file_name)

print(f'Cross-validation {len(names)} models required a duration of {time.time()-start:.2f} seconds')

Cross-validation 8 models required a duration of 53.11 seconds


In [34]:
print_evaluation_results(results, names)

LogisticRegression             	: 91.040% ( 1.414% )
KNN                            	: 92.015% ( 1.224% )
Decision tree                  	: 91.120% ( 1.486% )
Random Forest                  	: 94.736% ( 0.953% )
SVM                            	: 93.034% ( 1.435% )
RegularNets                    	: 92.449% ( 1.391% )
LDA                            	: 89.338% ( 1.801% )
Gaussian Naive Bayes           	: 69.725% ( 2.276% )


In [35]:
fig = plot_evaluation_boxplots(results, names, title = 'Models Performance On One-Hot Encoded "apistats" Data', y_axis = 'Accuracy')

figure_path = 'figures/evaluation_onehot_apistats_data.html'

save_figures_to_html(figure_path, [fig])

IFrame(figure_path, width=900, height=600)

### 1. apistats (count encoded)

In [7]:
file_name = 'count_encoded_apistats_dataset.csv'

preview_data(folder_path, file_name)

Nb Observations: 3822
Nb Columns: 305


,LdrUnloadDll,RegCloseKey,GetSystemTimeAsFileTime,LoadStringW,GetSystemInfo,RegQueryValueExA,LdrGetProcedureAddress,MessageBoxTimeoutW,RegSetValueExA,NtTerminateProcess,...,CDocument_write,WSAConnect,ExitWindowsEx,CopyFileExW,NtDeleteFile,CreateServiceW,WNetGetProviderNameW,RtlCreateUserProcess,NtLoadKeyEx,NtLoadDriver
md5,,,,,,,,,,,,,,,,,,,,,
00263ca2071dc9a6ee577eb356b0d1d9,4,3,1,2,1,13,6,1,6,3,...,0,0,0,0,0,0,0,0,0,0
0137996cd3aa197ae8eb64fef12c044a,0,0,6,2,1,0,55,1,0,0,...,0,0,0,0,0,0,0,0,0,0
0282f83bbfb58c08b54dbd8015e54d2e,0,9,1,0,0,0,21,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
start = time.time()

names, results = get_evaluation_results(folder_path, file_name)

print(f'Cross-validation on {len(names)} models required a duration of {time.time()-start:.2f} seconds')

Cross-validation on 8 models required a duration of 53.44 seconds


In [9]:
print_evaluation_results(results, names)

LogisticRegression             	: 71.961% ( 1.777% )
KNN                            	: 86.168% ( 1.706% )
Decision tree                  	: 91.009% ( 1.579% )
Random Forest                  	: 94.828% ( 1.143% )
SVM                            	: 59.524% ( 0.464% )
RegularNets                    	: 88.139% ( 1.942% )
LDA                            	: 82.156% ( 1.641% )
Gaussian Naive Bayes           	: 56.550% ( 1.813% )


In [10]:
fig = plot_evaluation_boxplots(results, names, title = 'Models Performance On Count Encoded "apistats" Data', y_axis = 'Accuracy')

figure_path = 'figures/evaluation_count_apistats_data.html'

save_figures_to_html(figure_path, [fig])

IFrame(figure_path, width=900, height=600)

### 3. dll_loaded (one-hot encoded)

In [11]:
file_name = 'onehot_encoded_dll_dataset.csv'

preview_data(folder_path, file_name)

Nb Observations: 3162
Nb Columns: 2242


,RTUTILS.DLL,kernel32.dll,label,kernel32,C:\Users\cucko\AppData\Local\Temp\emedcfd.dll,RpcRtRemote.dll,CRYPTSP.dll,CLBCatQ.DLL,msimg32.dll,libssl32.dll,...,C:\Users\cucko\AppData\Local\Temp\VmX.dll,C:\Users\cucko\AppData\Local\Temp\VirusShare_1d4457e8e6917937845f55ebbce2fc49.dll,C:\Users\cucko\AppData\Local\Temp\VirusShare_1d50b69a05d60d4f9f703b789a2933de.dll,C:\Users\cucko\AppData\Local\Temp\VirusShare_268eef019bf65b2987e945afaf29643f.dll,C:\Users\cucko\AppData\Local\Temp\VirusShare_3c1b2fabb7d74bc5be0820eae4107f8a.exe,C:\Users\cucko\AppData\Local\Temp\VirusShare_43b844c35e1a933e9214588be81ce772.dll,C:\Users\cucko\AppData\Local\Temp\VirusShare_933b11bc4799f8d9f65466fb2e3ea659.exe,C:\Users\cucko\AppData\Local\Temp\VirusShare_9fc3ed6c9b8056fbf155f79569ca7cb1.exe,D3DIM700.DLL,C:\Users\cucko\AppData\Local\Temp\VirusShare_bcbdef1678049378be04719ed29078d2.dll
md5,,,,,,,,,,,,,,,,,,,,,
00263ca2071dc9a6ee577eb356b0d1d9,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0137996cd3aa197ae8eb64fef12c044a,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0282f83bbfb58c08b54dbd8015e54d2e,0,0,0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
start = time.time()

names, results = get_evaluation_results(folder_path, file_name)

print(f'Cross-validation {len(names)} models required a duration of {time.time()-start:.2f} seconds')

Cross-validation 8 models required a duration of 207.83 seconds


In [13]:
print_evaluation_results(results, names)

LogisticRegression             	: 86.959% ( 2.104% )
KNN                            	: 84.419% ( 1.980% )
Decision tree                  	: 84.893% ( 1.961% )
Random Forest                  	: 86.960% ( 1.766% )
SVM                            	: 87.528% ( 1.928% )
RegularNets                    	: 86.601% ( 2.104% )
LDA                            	: 82.046% ( 2.454% )
Gaussian Naive Bayes           	: 64.895% ( 3.177% )


In [14]:
fig = plot_evaluation_boxplots(results, names, title = 'Models Performance On One-Hot Encoded "dll_loaded" Data', y_axis = 'Accuracy')

figure_path = 'figures/evaluation_onehot_dll_data.html'

save_figures_to_html(figure_path, [fig])

IFrame(figure_path, width=900, height=600)